In [1]:
import tkinter as tk
from tkinter import filedialog, ttk, messagebox
from PIL import Image, ImageOps, ImageTk, ImageEnhance
import customtkinter as ctk
import cv2
import numpy as np
from scipy.signal import convolve2d
from scipy.ndimage import minimum_filter, maximum_filter, median_filter
from skimage.filters import roberts, sobel, scharr, prewitt
import tensorflow as tf



# Initialize the root window
root = ctk.CTk()  # Use CTk for the root window
root.geometry("1920x1080")
root.title("Image Manipulation Tool")
root.config(bg="#f0f0f0")

pen_color = "black"
pen_size = 5
file_path = ""
image_ref_original = None
image_ref_modified = None

# Store the original and modified images
original_image = None
modified_image = None



# Function to add an image and display it
def add_image():
    global file_path, original_image, image_ref_original, modified_image, image_ref_modified
    file_path = filedialog.askopenfilename(title="Select Image", filetypes=[("Image Files", "*.jpg;*.jpeg;*.png")])
    if file_path:
        original_image = Image.open(file_path)
        width, height = int(original_image.width / 2), int(original_image.height / 2)
        original_image = original_image.resize((width, height), Image.Resampling.LANCZOS)
        modified_image = original_image.copy()  # Create a copy for manipulation
        image_ref_original = ImageTk.PhotoImage(original_image)
        image_ref_modified = ImageTk.PhotoImage(modified_image)

        canvas_original.config(width=width, height=height)
        canvas_modified.config(width=width, height=height)
        
        canvas_original.create_image(0, 0, image=image_ref_original, anchor="nw")
        canvas_modified.create_image(0, 0, image=image_ref_modified, anchor="nw")

# Style the input dialog with a custom theme
def styled_input_dialog(title, prompt):
    dialog_window = ctk.CTkToplevel(root)  # Use CTkToplevel for the dialog window
    dialog_window.title(title)
    
    
    dialog_window.grab_set()

    
    window_width, window_height = 300, 150
    screen_width = root.winfo_screenwidth()
    screen_height = root.winfo_screenheight()
    x_cordinate = int((screen_width/2) - (window_width/2))
    y_cordinate = int((screen_height/2) - (window_height/2))
    dialog_window.geometry(f"{window_width}x{window_height}+{x_cordinate}+{y_cordinate}")
    
    dialog_window.configure(bg_color="#4A8")  
    prompt_label = ctk.CTkLabel(dialog_window, text=prompt, font=("Helvetica", 12), text_color="white")
    prompt_label.pack(pady=10)

    input_entry = ctk.CTkEntry(dialog_window, font=("Helvetica", 12))
    input_entry.pack(pady=10)

    def on_submit():
        dialog_window.user_input = input_entry.get()
        dialog_window.destroy()

    submit_button = ctk.CTkButton(dialog_window, text="Submit", command=on_submit, font=("Helvetica", 12), text_color="white", fg_color="#333")
    submit_button.pack(pady=10)

    root.wait_window(dialog_window)  # Wait until dialog is closed

    return getattr(dialog_window, 'user_input', None)


# Rotate the image
def rotate_image():
    global modified_image, image_ref_modified
    angle = styled_input_dialog("Input", "Enter rotation angle:")
    if angle is not None:
        modified_image = modified_image.rotate(float(angle), expand=True)
        image_ref_modified = ImageTk.PhotoImage(modified_image)
        canvas_modified.config(width=modified_image.width, height=modified_image.height)
        canvas_modified.create_image(0, 0, image=image_ref_modified, anchor="nw")


# Crop the image
def crop_image():
    global modified_image, image_ref_modified
    start_x, start_y = 50, 50  # Sample crop area for demonstration
    end_x, end_y = modified_image.width - 50, modified_image.height - 50
    modified_image = modified_image.crop((start_x, start_y, end_x, end_y))
    image_ref_modified = ImageTk.PhotoImage(modified_image)
    canvas_modified.config(width=modified_image.width, height=modified_image.height)
    canvas_modified.create_image(0, 0, image=image_ref_modified, anchor="nw")
    
def invert_colors():
    global modified_image, image_ref_modified
    if modified_image is not None:
        # Convert the image to a numpy array for pixel manipulation
        array_image = np.array(modified_image)

        # Invert the colors using numpy (255 - pixel value)
        inverted_image = 255 - array_image

        # Convert back to PIL format
        inverted_image_pil = Image.fromarray(inverted_image)

        # Update the modified image and display it
        image_ref_modified = ImageTk.PhotoImage(inverted_image_pil)
        canvas_modified.config(width=inverted_image_pil.width, height=inverted_image_pil.height)
        canvas_modified.create_image(0, 0, image=image_ref_modified, anchor="nw")
        
        # Update the modified image to reflect the inverted colors
        modified_image = inverted_image_pil

#grayscale the image

def convert_to_grayscale():
    global modified_image, image_ref_modified
    if modified_image is not None:
        # Convert the image to grayscale using PIL's convert method
        grayscale_image = modified_image.convert("L")  # "L" mode is for grayscale
        
        image_ref_modified = ImageTk.PhotoImage(grayscale_image)
        canvas_modified.config(width=grayscale_image.width, height=grayscale_image.height)
        canvas_modified.create_image(0, 0, image=image_ref_modified, anchor="nw")
        
        # Update the modified image to reflect grayscale
        modified_image = grayscale_image


#gaussian image
def gaussian_filter():
    global modified_image, image_ref_modified
    if modified_image is not None:
        array_image = np.array(modified_image)
        
        # Define the Gaussian filter
        gaussian_filter = np.array([[1, 4, 7, 4, 1], 
                                    [4, 16, 26, 16, 4], 
                                    [7, 26, 41, 26, 7],
                                    [4, 16, 26, 16, 4],
                                    [1, 4, 7, 4, 1]], dtype=np.float32) / 273
        
        # Apply the Gaussian filter using cv2
        filtered_gaussian_image = cv2.filter2D(array_image, -1, gaussian_filter)
        
        # Convert filtered image back to PIL format
        filtered_image = Image.fromarray(filtered_gaussian_image)
        
        image_ref_modified = ImageTk.PhotoImage(filtered_image)  # Store the reference
        canvas_modified.config(width=filtered_image.width, height=filtered_image.height)
        canvas_modified.create_image(0, 0, image=image_ref_modified, anchor="nw")

#median filter

def median_filter():
    global modified_image, image_ref_modified
    if modified_image is not None:
        array_image = np.array(modified_image)

        # Apply the Median filter using cv2
        median_filtered_image = cv2.medianBlur(array_image, 3)
    
        # Convert the filtered image back to PIL format
        median_image = Image.fromarray(median_filtered_image)
    
        image_ref_modified = ImageTk.PhotoImage(median_image)  # Store the reference
        canvas_modified.config(width=median_image.width, height=median_image.height)
        canvas_modified.create_image(0, 0, image=image_ref_modified, anchor="nw")

# Roberts filter
def roberts_filter():
    global modified_image, image_ref_modified
    if modified_image is not None:
        array_image = np.array(modified_image)

        kernel = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])
        roberts_filtered_image = cv2.filter2D(array_image, -1, kernel)
        
        roberts_image = Image.fromarray(roberts_filtered_image)
        
        image_ref_modified = ImageTk.PhotoImage(roberts_image)  # Store the reference
        canvas_modified.config(width=roberts_image.width, height=roberts_image.height)
        canvas_modified.create_image(0, 0, image=image_ref_modified, anchor="nw")

# Sharpen image
def sharpen_image():
    global modified_image, image_ref_modified
    if modified_image is not None:
        array_image = np.array(modified_image)

        # Define a sharpening kernel
        sharpening_kernel = np.array([[0, -1, 0], 
                                      [-1, 5, -1], 
                                      [0, -1, 0]])
        
        sharpened_image = cv2.filter2D(array_image, -1, sharpening_kernel)
    
        # Convert the sharpened image back to PIL format
        sharpened_image = Image.fromarray(sharpened_image)
    
        image_ref_modified = ImageTk.PhotoImage(sharpened_image)  # Store the reference
        canvas_modified.config(width=sharpened_image.width, height=sharpened_image.height)
        canvas_modified.create_image(0, 0, image=image_ref_modified, anchor="nw")
        

# Adjust brightness
def adjust_brightness():
    global modified_image, image_ref_modified
    brightness = styled_input_dialog("Input", "Enter brightness level (0.0 to 10.0):")
    if brightness is not None:
        brightness = float(brightness)
        enhancer = ImageEnhance.Brightness(modified_image)
        modified_image = enhancer.enhance(brightness)
        image_ref_modified = ImageTk.PhotoImage(modified_image)
        canvas_modified.create_image(0, 0, image=image_ref_modified, anchor="nw")

# Adjust contrast
def adjust_contrast():
    global modified_image, image_ref_modified
    contrast = styled_input_dialog("Input", "Enter contrast level (0.0 to 10.0):")
    if contrast is not None:
        contrast = float(contrast)
        enhancer = ImageEnhance.Contrast(modified_image)
        modified_image = enhancer.enhance(contrast)
        image_ref_modified = ImageTk.PhotoImage(modified_image)
        canvas_modified.create_image(0, 0, image=image_ref_modified, anchor="nw")

# Adjust color balance
def adjust_color_balance():
    global modified_image, image_ref_modified
    color_factor = styled_input_dialog("Input", "Enter color factor (0.0 to 10.0):")
    if color_factor is not None:
        color_factor = float(color_factor)
        enhancer = ImageEnhance.Color(modified_image)
        modified_image = enhancer.enhance(color_factor)
        image_ref_modified = ImageTk.PhotoImage(modified_image)
        canvas_modified.create_image(0, 0, image=image_ref_modified, anchor="nw")

# Save the modified image
def save_image():
    if modified_image is not None:
        save_path = filedialog.asksaveasfilename(defaultextension=".png", filetypes=[("PNG files", "*.png"), ("JPEG files", "*.jpg"), ("All files", "*.*")])
        if save_path:
            modified_image.save(save_path)
            messagebox.showinfo("Image Saved", f"Image saved successfully to {save_path}")
    else:
        messagebox.showwarning("No Image", "No modified image to save.")

# Reset the image to its original state
def reset_image():
    global modified_image, image_ref_modified
    modified_image = original_image.copy()
    image_ref_modified = ImageTk.PhotoImage(modified_image)
    canvas_modified.create_image(0, 0, image=image_ref_modified, anchor="nw")




#Segmentation

def kmeans_segmentation():
    global modified_image, image_ref_modified
    if modified_image is not None:
        array_image = np.array(modified_image)
        pixel_vals = array_image.reshape((-1, 3))
        pixel_vals = np.float32(pixel_vals)

        # Define criteria, number of clusters(K) and apply KMeans
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)
        k = 3  # You can adjust this for different segmentation results
        _, labels, centers = cv2.kmeans(pixel_vals, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
        centers = np.uint8(centers)
        segmented_image = centers[labels.flatten()]
        segmented_image = segmented_image.reshape(array_image.shape)

        kmeans_image = Image.fromarray(segmented_image)
        image_ref_modified = ImageTk.PhotoImage(kmeans_image)
        canvas_modified.config(width=kmeans_image.width, height=kmeans_image.height)
        canvas_modified.create_image(0, 0, image=image_ref_modified, anchor="nw")

# GrabCut Segmentation
def grabcut_segmentation():
    global modified_image, image_ref_modified
    if modified_image is not None:
        array_image = np.array(modified_image)
        mask = np.zeros(array_image.shape[:2], np.uint8)
        
        bgdModel = np.zeros((1, 65), np.float64)
        fgdModel = np.zeros((1, 65), np.float64)

        rect = (50, 50, array_image.shape[1] - 100, array_image.shape[0] - 100)
        cv2.grabCut(array_image, mask, rect, bgdModel, fgdModel, 5, cv2.GC_INIT_WITH_RECT)
        mask2 = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')
        grabcut_image = array_image * mask2[:, :, np.newaxis]

        grabcut_image = Image.fromarray(grabcut_image)
        image_ref_modified = ImageTk.PhotoImage(grabcut_image)
        canvas_modified.config(width=grabcut_image.width, height=grabcut_image.height)
        canvas_modified.create_image(0, 0, image=image_ref_modified, anchor="nw")


# Canny Edge Detection
def canny_edge_detection():
    global modified_image, image_ref_modified
    if modified_image is not None:
        array_image = np.array(modified_image)
        edges = cv2.Canny(array_image, 100, 200)

        canny_image = Image.fromarray(edges)
        image_ref_modified = ImageTk.PhotoImage(canny_image)
        canvas_modified.config(width=canny_image.width, height=canny_image.height)
        canvas_modified.create_image(0, 0, image=image_ref_modified, anchor="nw")

# Thresholding Segmentation
def thresholding_segmentation():
    global modified_image, image_ref_modified
    if modified_image is not None:
        array_image = np.array(modified_image.convert("L"))  # Convert to grayscale
        _, thresholded_image = cv2.threshold(array_image, 128, 255, cv2.THRESH_BINARY)

        thresholded_image_pil = Image.fromarray(thresholded_image)
        image_ref_modified = ImageTk.PhotoImage(thresholded_image_pil)
        canvas_modified.config(width=thresholded_image_pil.width, height=thresholded_image_pil.height)
        canvas_modified.create_image(0, 0, image=image_ref_modified, anchor="nw")

# Watershed Segmentation
def watershed_segmentation():
    global modified_image, image_ref_modified
    if modified_image is not None:
        array_image = np.array(modified_image.convert("RGB"))

        # Convert the image to grayscale
        gray = cv2.cvtColor(array_image, cv2.COLOR_RGB2GRAY)
        
        # Apply threshold
        ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
        
        # Noise removal
        kernel = np.ones((3, 3), np.uint8)
        opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=2)
        
        # Sure background area
        sure_bg = cv2.dilate(opening, kernel, iterations=3)
        
        # Finding sure foreground area
        dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
        ret, sure_fg = cv2.threshold(dist_transform, 0.7 * dist_transform.max(), 255, 0)
        
        # Finding unknown region
        sure_fg = np.uint8(sure_fg)
        unknown = cv2.subtract(sure_bg, sure_fg)
        
        # Marker labelling
        ret, markers = cv2.connectedComponents(sure_fg)
        markers = markers + 1
        
        # Mark the region of unknown with zero
        markers[unknown == 0] = 0
        
        markers = cv2.watershed(array_image, markers)
        array_image[markers == -1] = [255, 0, 0]  # Mark the boundaries
        
        watershed_image_pil = Image.fromarray(array_image)
        image_ref_modified = ImageTk.PhotoImage(watershed_image_pil)
        canvas_modified.config(width=watershed_image_pil.width, height=watershed_image_pil.height)
        canvas_modified.create_image(0, 0, image=image_ref_modified, anchor="nw")

#------------------DEEP LEARNING STYLE TRANSFER

def tensor_to_image(tensor):
  
    tensor_shape = tf.shape(tensor)
    number_elem_shape = tf.shape(tensor_shape)
    if number_elem_shape > 3:
        assert tensor_shape[0] == 1
        tensor = tensor[0]
    return tf.keras.preprocessing.image.array_to_img(tensor) 

# Function to load an image 
def load_img(path_to_img):
    
    max_dim = 512
    image = tf.io.read_file(path_to_img)
    image = tf.image.decode_jpeg(image)
    image = tf.image.convert_image_dtype(image, tf.float32) 

    shape = tf.shape(image)[:-1]
    shape = tf.cast(shape, tf.float32)  
    long_dim = max(shape)
    scale = max_dim / long_dim

    new_shape = tf.cast(shape * scale, tf.int32) 

    image = tf.image.resize(image, new_shape)
    image = image[tf.newaxis, :]  
    return image

def clip_image_values(image, min_value=0.0, max_value=255.0):
    """Clips the pixel values of the image to stay within the valid range."""
    return tf.clip_by_value(image, clip_value_min=min_value, clip_value_max=max_value)

def preprocess_image(image):
    '''Preprocesses the image to match the input requirements of the VGG19 model'''
    image = tf.cast(image, tf.float32)  # Cast image to float32
    image = tf.keras.applications.vgg19.preprocess_input(image)  
    return image

style_layers = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1']
content_layers = ['block5_conv2']
output_layers = style_layers + content_layers
NUM_STYLE_LAYERS = len(style_layers)
NUM_CONTENT_LAYERS = len(content_layers)

def vgg_model(layer_names):
    '''Creates a VGG model that returns the outputs of specific layers'''
    vgg = tf.keras.applications.VGG19(include_top=False, weights='imagenet')
    vgg.trainable = False
    outputs = [vgg.get_layer(name).output for name in layer_names]
    model = tf.keras.Model([vgg.input], outputs)
    return model

vgg = vgg_model(output_layers)

def get_style_loss(features, targets, weight=1.0):
    style_loss = tf.reduce_mean(tf.square(features - targets))
    return weight * style_loss

def get_content_loss(features, targets, weight=1.0):
    content_loss = tf.reduce_sum(tf.square(features - targets))
    return weight * content_loss

def gram_matrix(input_tensor):
    gram = tf.linalg.einsum('bijc,bijd->bcd', input_tensor, input_tensor)
    input_shape = tf.shape(input_tensor)
    height = input_shape[1]
    width = input_shape[2]
    num_locations = tf.cast(height * width, tf.float32)
    scaled_gram = gram / num_locations
    return scaled_gram

def get_style_image_features(image, vgg_model):
    preprocessed_style_image = preprocess_image(image)
    outputs = vgg_model(preprocessed_style_image)
    style_outputs = outputs[:NUM_STYLE_LAYERS]
    gram_style_features = [gram_matrix(style_output) for style_output in style_outputs]
    return gram_style_features

def get_content_image_features(image, vgg_model):
    preprocessed_content_image = preprocess_image(image)
    outputs = vgg_model(preprocessed_content_image)
    content_outputs = outputs[NUM_STYLE_LAYERS:]
    return content_outputs

def get_style_content_loss(style_targets, style_outputs, content_targets, content_outputs, style_weight, content_weight):
    style_loss = tf.add_n([get_style_loss(style_output, style_target, style_weight) 
                           for style_output, style_target in zip(style_outputs, style_targets)])
    
    content_loss = tf.add_n([get_content_loss(content_output, content_target, content_weight) 
                             for content_output, content_target in zip(content_outputs, content_targets)])
    
    style_loss = style_loss * style_weight / NUM_STYLE_LAYERS
    content_loss = content_loss * content_weight / NUM_CONTENT_LAYERS
    
    total_loss = style_loss + content_loss
    return total_loss

def calculate_gradients(image, style_targets, content_targets, style_weight, content_weight, var_weight, vgg_model):
    with tf.GradientTape() as tape:
        style_features = get_style_image_features(image, vgg_model)
        content_features = get_content_image_features(image, vgg_model)
        loss = get_style_content_loss(style_targets, style_features, content_targets, content_features, style_weight, content_weight)
        gradients = tape.gradient(loss, image)
    return gradients

def update_image_with_style(image, style_targets, content_targets, style_weight, var_weight, content_weight, optimizer, vgg_model):
    gradients = calculate_gradients(image, style_targets, content_targets, style_weight, content_weight, var_weight, vgg_model)
    optimizer.apply_gradients([(gradients, image)])
    image.assign(clip_image_values(image, min_value=0.0, max_value=255.0))

def fit_style_transfer(style_image, content_image, style_weight=1e-2, content_weight=1e-4, var_weight=0, optimizer=None, epochs=1, steps_per_epoch=1, vgg=None, display_fn=None):
    images = []
    step = 0

    if optimizer is None:
        optimizer = tf.optimizers.Adam(
            tf.keras.optimizers.schedules.ExponentialDecay(
                initial_learning_rate=20.0,
                decay_steps=100,
                decay_rate=0.5
            )
        )

    if vgg is None:
        raise ValueError("VGG model must be provided.")

    style_targets = get_style_image_features(style_image, vgg)
    content_targets = get_content_image_features(content_image, vgg)

    generated_image = tf.cast(content_image, dtype=tf.float32)
    generated_image = tf.Variable(generated_image)

    images.append(content_image)

    for n in range(epochs):
        for m in range(steps_per_epoch):
            step += 1
            update_image_with_style(generated_image, style_targets, content_targets, style_weight, var_weight, content_weight, optimizer, vgg)
            print(".", end="")
            if (m + 1) % 10 == 0:
                images.append(generated_image.numpy())

        if display_fn:
            display_image = tensor_to_image(generated_image)
            display_fn(display_image)

        images.append(generated_image.numpy())
        print(f"Train step: {step}")

    generated_image = tf.cast(generated_image, dtype=tf.float32)
    return generated_image, images

def select_image(image_type, window_context):
   
    filepath = filedialog.askopenfilename()
    if filepath:
        img = Image.open(filepath)
        img = img.resize((150, 150), Image.Resampling.LANCZOS)

        if image_type == 'content':
            window_context['content_image_path'].set(filepath)
            window_context['content_img_label'].imgtk = ImageTk.PhotoImage(img)
            window_context['content_img_label'].configure(image=window_context['content_img_label'].imgtk)
            window_context['content_img_label'].imgtk = window_context['content_img_label'].imgtk  

        elif image_type == 'style':
            window_context['style_image_path'].set(filepath)
            window_context['style_img_label'].imgtk = ImageTk.PhotoImage(img)
            window_context['style_img_label'].configure(image=window_context['style_img_label'].imgtk)
            window_context['style_img_label'].imgtk = window_context['style_img_label'].imgtk  


def start_style_transfer(window_context):
   
    content_image = load_img(window_context['content_image_path'].get())
    style_image = load_img(window_context['style_image_path'].get())

    style_weight = 2e-2
    content_weight = 1e-2

    adam = tf.optimizers.Adam(
        tf.keras.optimizers.schedules.ExponentialDecay(
            initial_learning_rate=20.0,
            decay_steps=100,
            decay_rate=0.50
        )
    )

    vgg = vgg_model(output_layers)

    output_image, _ = fit_style_transfer(
        style_image=style_image,
        content_image=content_image,
        style_weight=style_weight,
        content_weight=content_weight,
        var_weight=0,
        optimizer=adam,
        epochs=10,
        steps_per_epoch=10,
        vgg=vgg
    )

    stylized_image_pil = tensor_to_image(output_image)
    stylized_image_pil = stylized_image_pil.resize((150, 150), Image.Resampling.LANCZOS)

    result_image_ctk = ImageTk.PhotoImage(stylized_image_pil)
    window_context['result_img_label'].configure(image=result_image_ctk)
    window_context['result_img_label'].imgtk = result_image_ctk  # Prevent garbage collection



#----------------DEEP LEARNING ENDS

#----deep learning gui begins
def open_style_transfer_window():
    style_window = ctk.CTkToplevel(root)
    style_window.geometry("1920x1080")
    style_window.title("Neural Style Transfer")

    # Create a context dictionary to pass around the necessary UI elements
    window_context = {
        'content_image_path': ctk.StringVar(),
        'style_image_path': ctk.StringVar(),
        'content_img_label': None,
        'style_img_label': None,
        'result_img_label': None,
    }

    title_label = ctk.CTkLabel(style_window, text="Neural Style Transfer", font=('Arial', 20))
    title_label.pack(pady=20)

    
    content_btn = ctk.CTkButton(style_window, text="Select Content Image", command=lambda: select_image('content', window_context))
    content_btn.pack(pady=10)
    window_context['content_img_label'] = ctk.CTkLabel(style_window, text="")
    window_context['content_img_label'].pack()

    
    style_btn = ctk.CTkButton(style_window, text="Select Style Image", command=lambda: select_image('style', window_context))
    style_btn.pack(pady=10)
    window_context['style_img_label'] = ctk.CTkLabel(style_window, text="")
    window_context['style_img_label'].pack()

    
    start_btn = ctk.CTkButton(style_window, text="Start Style Transfer", command=lambda: start_style_transfer(window_context))
    start_btn.pack(pady=20)

    window_context['result_img_label'] = ctk.CTkLabel(style_window, text="Result will be displayed here")
    window_context['result_img_label'].pack()

#--- deep learning gui ends
        

# UI Elements
left_frame = ctk.CTkScrollableFrame(root, width=200, height=600, corner_radius=0, bg_color="#333333")  
left_frame.pack(side="left", fill="y")

canvas_original = ctk.CTkCanvas(root, width=500, height=600, bg="#ffffff")  
canvas_original.pack(side="left", padx=10, pady=10)

canvas_modified = ctk.CTkCanvas(root, width=500, height=600, bg="#ffffff")  
canvas_modified.pack(side="right", padx=10, pady=10)

# Add image button
add_icon = ctk.CTkImage(dark_image=Image.open("icons/upload.png"), size=(20, 20))     
image_button = ctk.CTkButton(left_frame, text="Add Image", command=add_image, image=add_icon)
image_button.pack(pady=20)



# Rotate image button
rotate_icon = ctk.CTkImage(dark_image=Image.open("icons/rotate.png"), size=(20, 20))
rotate_button = ctk.CTkButton(left_frame, text="Rotate Image", command=rotate_image, image=rotate_icon)
rotate_button.pack(pady=10)


# Crop image button
crop_icon = ctk.CTkImage(dark_image=Image.open("icons/crop.png"), size=(20, 20))
crop_button = ctk.CTkButton(left_frame, text="Crop Image", command=crop_image, image=crop_icon)
crop_button.pack(pady=10)

# Invert colors button
invert_icon = ctk.CTkImage(dark_image=Image.open("icons/invert.png"), size=(20, 20))  # Assuming you have an invert icon
invert_button = ctk.CTkButton(left_frame, text="Invert Colors", command=invert_colors, image=invert_icon)
invert_button.pack(pady=10)


# Adjust brightness button
brightness_icon = ctk.CTkImage(dark_image=Image.open("icons/brightness.png"), size=(20, 20))
brightness_button = ctk.CTkButton(left_frame, text="Adjust Brightness", command=adjust_brightness, image=brightness_icon)
brightness_button.pack(pady=10)




#Grayscale image button
grayscale_icon = ctk.CTkImage(dark_image=Image.open("icons/grayscale.png"), size=(20, 20))  # If you have a grayscale icon
grayscale_button = ctk.CTkButton(left_frame, text="Convert to Grayscale", command=convert_to_grayscale, image=grayscale_icon)
grayscale_button.pack(pady=10)




#gaussian filter
gaussian_icon = ctk.CTkImage(dark_image=Image.open("icons/gaussian.png"), size=(20, 20))
gaussian_filter_button = ctk.CTkButton(left_frame, text="Gaussian Filter", command=gaussian_filter, image=gaussian_icon)
gaussian_filter_button.pack(pady=10)

#median filter
median_icon = ctk.CTkImage(dark_image=Image.open("icons/graph.png"), size=(20, 20))
median_filter_button =ctk.CTkButton(left_frame, text="Median Blur", command=median_filter, image=median_icon)
median_filter_button.pack(pady=10)

#roberts filter
roberts_filter_button =ctk.CTkButton(left_frame, text="Roberts Image", command=roberts_filter)
roberts_filter_button.pack(pady=10)

#sharpen image
sharpen_icon = ctk.CTkImage(dark_image=Image.open("icons/sharpen.png"), size=(20, 20))
sharpen_button =ctk.CTkButton(left_frame, text="Sharpen Image", command=sharpen_image,image=sharpen_icon)
sharpen_button.pack(pady=10)

# Adjust contrast button
contrast_icon = ctk.CTkImage(dark_image=Image.open("icons/contrast.png"), size=(20, 20))
contrast_button = ctk.CTkButton(left_frame, text="Adjust Contrast", command=adjust_contrast, image=contrast_icon)
contrast_button.pack(pady=10)

# Adjust color balance button
filter_icon = ctk.CTkImage(dark_image=Image.open("icons/filter.png"), size=(20, 20))
color_balance_button = ctk.CTkButton(left_frame, text="Adjust Color Balance", command=adjust_color_balance, image=filter_icon)
color_balance_button.pack(pady=10)




# Add buttons for segmentation techniques
kmeans_button = ctk.CTkButton(left_frame, text="K-Means Segmentation", command=kmeans_segmentation)
kmeans_button.pack(pady=10)

grabcut_button = ctk.CTkButton(left_frame, text="GrabCut Segmentation", command=grabcut_segmentation)
grabcut_button.pack(pady=10)

canny_button = ctk.CTkButton(left_frame, text="Canny Edge Detection", command=canny_edge_detection)
canny_button.pack(pady=10)

threshold_button = ctk.CTkButton(left_frame, text="Thresholding Segmentation", command=thresholding_segmentation)
threshold_button.pack(pady=10)

watershed_button = ctk.CTkButton(left_frame, text="Watershed Segmentation", command=watershed_segmentation)
watershed_button.pack(pady=10)

#----deep larning 

open_style_transfer_btn = ctk.CTkButton(left_frame, text="Open Neural Style Transfer", command=open_style_transfer_window)
open_style_transfer_btn.pack(pady=20)




# Reset image button
reset_icon = ctk.CTkImage(dark_image=Image.open("icons/undo.png"), size=(20, 20))
reset_button = ctk.CTkButton(left_frame, text="Reset Image", command=reset_image, image=reset_icon)
reset_button.pack(pady=20)

# Save image button
save_icon = ctk.CTkImage(dark_image=Image.open("icons/save.png"), size=(20, 20))
save_button = ctk.CTkButton(left_frame, text="Save Image", command=save_image, image=save_icon)
save_button.pack(pady=20)




root.mainloop()

